# P. Visión

In [2]:
import numpy as np
import cv2
import os
import random
from skimage.feature import hog
from skimage import exposure

In [3]:
os.chdir('C:\\Users\\alumno\\Desktop\\train_pos')
listaPos = os.listdir()
fotosPosTrain = {}
for i in range(len(listaPos)):
    fotosPosTrain[i] = cv2.imread(listaPos[i],0)

In [5]:
os.chdir('C:\\Users\\alumno\\Desktop\\train_neg')
listaNeg = os.listdir()
fotosNegTrainSinTratar = {}
for i in range(len(listaNeg)):
    fotosNegTrainSinTratar[i] = cv2.imread(listaNeg[i],0)


In [14]:
numeroAleatoriasPorFoto = 3
rowsFotoPersona = fotosPosTrain[0].shape[0]
colsFotoPersona = fotosPosTrain[0].shape[1]

rowsFotoEntorno = fotosNegTrain[0].shape[0]
colsFotoEntorno = fotosNegTrain[0].shape[1]

fotosNegTrain = {}

aux = 0
for i in range(len(listaNeg)):
    
    for j in range(numeroAleatoriasPorFoto):
        randRow = random.randint(0,rowsFotoEntorno - rowsFotoPersona)
        randCol = random.randint(0,colsFotoEntorno - colsFotoPersona)
        randomImage = fotosNegTrain[i][randRow:randRow + rowsFotoPersona,randCol: randCol + colsFotoPersona]
        fotosNegTrain.add(aux,randomImage)
        aux = aux + 1
    
cv2.imshow('Original',fotosNegTrain[0])
cv2.imshow('Random',randomImage)
cv2.waitKey(0)
cv2.destroyAllWindows()

(134, 70)


In [67]:
nFilasFoto,nColsFoto = fotosPosTrain[0].shape
print(nFilasFoto/2)

67.0


In [110]:
fdPosTrain = np.zeros((len(fotosPosTrain),6804))
for i in range(len(fotosPosTrain)):
#fd,imHog = hog(fotosPosTrain[1], orientations=8, pixels_per_cell=(8, 8),cells_per_block=(3, 3),visualize=True)
    fdPosTrain[i] = hog(fotosPosTrain[i])
    #imHog_re = exposure.rescale_intensity(imHog, in_range = (0,10))
print(fdPosTrain.shape,fotosPosTrain[1].shape,134*70)

(563, 6804) (134, 70) 9380


In [6]:
# fdNegTrain = np.zeros((len(fotosNegTrain),6804))
# for i in range(len(fotosPosTrain)):
#     fdNegTrain[i] = hog(fotosPosTrain[i])
fdTest,imHogTest = hog(fotosNegTrain[1], orientations=9, pixels_per_cell=(8, 8),cells_per_block=(3, 3),visualize=True)
print(fdTest.shape,imHogTest.shape)

(86184,) (240, 320)


In [5]:
cv2.imshow("sadas",imHogTest)
cv2.imshow("ds2da",fotosNegTrain[1])
cv2.waitKey(0)
cv2.destroyAllWindows()

In [85]:
print()

[0.30046368 0.14791104 0.10443662 ... 0.12432016 0.09187911 0.03539276]
